<a href="https://colab.research.google.com/github/johnathanacortesd/Zero_Entreno_Roberta_Mpnet/blob/main/Zero_Entreno_Roberta_Mpnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 📊 Interfaz para Concatenar Columnas
# --- Importaciones necesarias ---
import pandas as pd
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, clear_output
import io

# --- Instalación silenciosa de la dependencia para .xlsx ---
!pip install -q openpyxl

# --- Clase principal de la aplicación ---
class VisualConcatenator:
    def __init__(self):
        """Inicializa la interfaz y sus componentes."""
        self.df = None
        self.output_df = None

        # --- Definición de los Widgets (Componentes de la UI) ---
        self.title = widgets.HTML("<h2>🔗 Concatenador Visual de Columnas</h2><hr>")
        self.instructions = widgets.HTML("<h4>Paso 1: Sube tu archivo Excel para empezar.</h4>")

        self.uploader = widgets.FileUpload(
            accept='.xlsx',
            description='Subir Archivo',
            button_style='primary'
        )

        # Contenedor para la segunda parte de la UI (que aparece después de subir)
        self.step2_container = widgets.VBox([])

        # Área de salida para mensajes y resultados
        self.output_area = widgets.Output()

        # --- Observador de eventos ---
        # Llama a la función _handle_upload cuando se sube un archivo
        self.uploader.observe(self._handle_upload, names='value')

    def _handle_upload(self, change):
        """Se activa al subir un archivo. Lee el archivo y crea la UI de selección."""
        # Limpia cualquier contenido previo en el área de salida
        with self.output_area:
            clear_output()

        uploaded_file_info = self.uploader.value
        if not uploaded_file_info:
            return # Si se cancela la subida, no hacer nada

        # Obtener el nombre y contenido del archivo subido
        filename = list(uploaded_file_info.keys())[0]
        content = uploaded_file_info[filename]['content']

        try:
            with self.output_area:
                print(f"⏳ Leyendo '{filename}'...")
                # Leer el archivo Excel desde la memoria
                self.df = pd.read_excel(io.BytesIO(content))
                print(f"✅ ¡Archivo cargado! Se encontraron {self.df.shape[0]} filas.")
                print(f"📊 Columnas disponibles: {', '.join(self.df.columns)}")

            # Si el archivo se leyó bien, crear la siguiente parte de la interfaz
            self._create_column_selector_ui()

        except Exception as e:
            with self.output_area:
                print(f"❌ Error al leer el archivo. Asegúrate de que es un .xlsx válido.")
                print(f"Detalle: {e}")

    def _create_column_selector_ui(self):
        """Crea los widgets para seleccionar columnas y procesar."""

        # Nuevas instrucciones para el usuario
        step2_instructions = widgets.HTML("""
        <h4>Paso 2: Selecciona las columnas a unir.</h4>
        <p><i>Usa <b>Ctrl+Click</b> (o <b>Cmd+Click</b> en Mac) para elegir varias.</i></p>
        """)

        # Widget para seleccionar múltiples columnas
        self.column_selector = widgets.SelectMultiple(
            options=self.df.columns.tolist(),
            description='Columnas:',
            layout=widgets.Layout(height='180px', width='95%'),
            style={'description_width': 'initial'}
        )

        # Botón para iniciar el procesamiento
        self.process_button = widgets.Button(
            description='🔗 Procesar y Descargar Resultado',
            button_style='success',
            icon='cogs',
            layout=widgets.Layout(width='300px', height='40px', margin='10px 0 0 0')
        )

        # Conectar el botón a la función de procesamiento
        self.process_button.on_click(self._process_and_download)

        # Actualizar el contenedor del Paso 2 con los nuevos widgets
        self.step2_container.children = [
            step2_instructions,
            self.column_selector,
            self.process_button
        ]

    def _process_and_download(self, b):
        """Realiza la concatenación, acortado y descarga del archivo."""
        with self.output_area:
            clear_output()

            selected_cols = self.column_selector.value
            if len(selected_cols) < 2:
                print("❌ Error: Debes seleccionar al menos dos columnas para poder unirlas.")
                return

            print("🚀 Procesando los datos...")
            print(f"   - Uniendo columnas: {', '.join(selected_cols)}")

            try:
                # --- Lógica principal ---
                # 1. Función para acortar el texto a 80 palabras
                def acortar_texto(texto, limite=80):
                    palabras = str(texto).split()
                    if len(palabras) > limite:
                        return ' '.join(palabras[:limite]) + '...'
                    return ' '.join(palabras) # .join() limpia espacios múltiples

                # 2. Convertir columnas a texto, rellenar vacíos y unir con un espacio
                texto_unido = self.df[list(selected_cols)].fillna('').astype(str).apply(
                    lambda fila: ' '.join(fila), axis=1
                )

                # 3. Acortar el texto ya unido
                texto_acortado = texto_unido.apply(acortar_texto)

                # 4. Crear el DataFrame final con una única columna llamada "resumen"
                self.output_df = pd.DataFrame({'resumen': texto_acortado})

                print("\n✅ ¡Procesamiento completado con éxito!")

                # --- Lógica de descarga ---
                output_filename = 'resumen_concatenado.xlsx'
                print(f"\n📦 Generando el archivo '{output_filename}' para descarga...")

                # Guardar el DataFrame en un archivo Excel dentro del entorno de Colab
                self.output_df.to_excel(output_filename, index=False, engine='openpyxl')

                # Usar la función de Colab para iniciar la descarga en el navegador
                files.download(output_filename)

            except Exception as e:
                print(f"\n❌ Ocurrió un error inesperado durante el procesamiento.")
                print(f"Detalle: {e}")

    def display_app(self):
        """Muestra la aplicación completa en la celda."""
        display(
            self.title,
            self.instructions,
            self.uploader,
            self.step2_container,
            self.output_area
        )

# --- Punto de entrada: Crear y mostrar la aplicación ---
app = VisualConcatenator()
app.display_app()

HTML(value='<h2>🔗 Concatenador Visual de Columnas</h2><hr>')

HTML(value='<h4>Paso 1: Sube tu archivo Excel para empezar.</h4>')

FileUpload(value={}, accept='.xlsx', button_style='primary', description='Subir Archivo')

VBox()

Output()

In [2]:
# @title 🔧 Instalaciones
print("Instalando dependencias...")
!pip install -q transformers sentence-transformers pandas openpyxl tqdm accelerate scikit-learn
print("Listo.")

Instalando dependencias...
Listo.


In [9]:
# @title 🚀 Analizador de Tono y Tema v15
# Roberta-large-multilingual-sentiment | paraphrase-multilingual-mpnet-base-v2
# ===============================================================
# MEJORAS IMPLEMENTADAS:
# 1. Agrupamiento semántico mejorado con clustering jerárquico
# 2. Sistema de votación para tono en grupos similares
# 3. Refinamiento de temas con análisis de keywords específicos
# 4. Cache de embeddings para optimización
# 5. Post-procesamiento de consistencia tema-tono
# ===============================================================

import os
import io
import re
import gc
import time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from google.colab import files
from collections import Counter, defaultdict
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from scipy.spatial.distance import cosine

# Configuración GPU
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.cuda.empty_cache()

    gpu_props = torch.cuda.get_device_properties(0)
    total_memory = gpu_props.total_memory / 1024**3
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ VRAM Total: {total_memory:.1f} GB")
else:
    device = torch.device("cpu")
    print("⚠️ Usando CPU. Recomendado activar GPU para mejor rendimiento")

# CONFIGURACIÓN OPTIMIZADA
MODELO_SENTIMIENTO = "clapAI/roberta-large-multilingual-sentiment"
MODELO_EMBEDDING = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
BATCH_SENT = 64
BATCH_EMB = 256
MAX_LEN_SENT = 256
MAX_CHARS_CLEAN = 600

# Parámetros de agrupamiento mejorados
SIMILARITY_THRESHOLD_STRICT = 0.92  # Para textos muy similares
SIMILARITY_THRESHOLD_MODERATE = 0.85  # Para textos relacionados
MIN_CLUSTER_SIZE = 2
PREFIX_WORDS = 8  # Aumentado para mejor discriminación

# DICCIONARIOS EXPANDIDOS Y MEJORADOS
TOPIC_KEYWORDS = {
    "Política y Gobierno": {
        "primary": [
            "presidente petro", "gustavo petro", "gobierno nacional", "congreso",
            "senado", "cámara representantes", "reforma tributaria", "reforma pensional",
            "ministro", "ministerio", "francia márquez", "gabinete", "decreto presidencial"
        ],
        "secondary": [
            "coalición", "oposición", "partido político", "movimiento", "elecciones",
            "campaña", "candidato", "votación", "referendo", "plebiscito", "consulta"
        ]
    },
    "Economía y Finanzas": {
        "primary": [
            "pib", "inflación", "banco república", "peso colombiano", "tasa cambio",
            "salario mínimo", "tributaria", "impuestos", "ecopetrol", "petróleo",
            "exportaciones", "crecimiento económico", "recesión", "desempleo"
        ],
        "secondary": [
            "empleo", "importaciones", "comercio", "inversión extranjera",
            "pymes", "industria", "producción", "mercado", "bolsa valores", "economía"
        ]
    },
    "Seguridad y Justicia": {
        "primary": [
            "policía nacional", "fiscalía", "homicidio", "asesinato", "captura",
            "eln", "disidencias farc", "paz total", "narcotráfico", "extorsión",
            "secuestro", "ejército nacional", "fuerzas militares", "guerrilla"
        ],
        "secondary": [
            "criminalidad", "hurto", "robo", "crimen organizado", "investigación",
            "corte suprema", "juzgado", "sentencia", "proceso judicial", "defensoría",
            "delito", "violencia", "inseguridad", "pandilla"
        ]
    },
    "Salud y Bienestar": {
        "primary": [
            "ministerio salud", "minsalud", "eps", "sistema salud", "reforma salud",
            "hospital", "clínica", "covid", "pandemia", "vacunación", "emergencia médica"
        ],
        "secondary": [
            "ips", "médico", "enfermera", "medicamentos", "tratamiento",
            "atención médica", "pacientes", "enfermedad", "cobertura", "salud pública",
            "prevención", "diagnóstico", "cirugía"
        ]
    },
    "Educación y Cultura": {
        "primary": [
            "ministerio educación", "mineducación", "universidades públicas",
            "icfes", "pruebas saber", "colegios", "educación superior", "becas"
        ],
        "secondary": [
            "estudiantes", "profesores", "maestros", "educación pública", "matrícula",
            "calendario académico", "investigación científica", "ciencia", "tecnología",
            "arte", "cultura", "patrimonio", "museo", "biblioteca"
        ]
    },
    "Infraestructura y Transporte": {
        "primary": [
            "carretera", "doble calzada", "túnel", "puente", "vía", "peaje",
            "transmilenio", "metro", "metro bogotá", "metro medellín", "aeropuerto eldorado"
        ],
        "secondary": [
            "avianca", "latam", "invías", "ani", "concesión vial", "obra pública",
            "construcción", "vivienda", "acueducto", "alcantarillado", "energía",
            "electricidad", "infraestructura", "transporte público"
        ]
    },
    "Medio Ambiente y Clima": {
        "primary": [
            "deforestación", "amazonas colombiana", "páramo", "cambio climático",
            "minambiente", "parques naturales", "biodiversidad", "especies amenazadas"
        ],
        "secondary": [
            "contaminación", "emisiones", "energías renovables", "minería ilegal",
            "carbón", "oro", "ríos", "agua", "desastre natural", "inundación",
            "sequía", "reforestación", "medio ambiente", "ecología", "sostenibilidad"
        ]
    },
    "Regiones y Territorio": {
        "primary": [
            "bogotá", "medellín", "cali", "barranquilla", "cartagena",
            "antioquia", "valle del cauca", "atlántico", "cundinamarca", "santander"
        ],
        "secondary": [
            "boyacá", "tolima", "huila", "cauca", "nariño", "córdoba", "bolívar",
            "meta", "municipio", "alcalde", "alcaldía", "gobernador", "gobernación",
            "concejo", "asamblea departamental"
        ]
    },
    "Social y Derechos Humanos": {
        "primary": [
            "protestas", "paro nacional", "manifestación", "derechos humanos",
            "víctimas conflicto", "desplazamiento forzado", "líderes sociales",
            "asesinato líderes", "amenazas", "masacre"
        ],
        "secondary": [
            "restitución tierras", "indígenas", "comunidades étnicas", "afrocolombianos",
            "campesinos", "onu", "violencia género", "feminicidio", "mujeres",
            "juventud", "adulto mayor", "población vulnerable", "discriminación"
        ]
    },
    "Deportes y Entretenimiento": {
        "primary": [
            "selección colombia", "james rodríguez", "luis díaz", "falcao",
            "liga betplay", "millonarios", "nacional", "ciclismo colombiano",
            "egan bernal", "nairo quintana"
        ],
        "secondary": [
            "fútbol colombiano", "américa cali", "junior barranquilla",
            "deportivo cali", "atlético nacional", "boxeo", "juegos olímpicos",
            "entretenimiento", "cine colombiano", "música", "vallenato", "festival"
        ]
    }
}

# Patrones de sentimiento contextual mejorados
SENTIMENT_PATTERNS = {
    "strong_negative": [
        r'\b(tragedia|desastre|crisis grave|escándalo|corrupción masiva|catástrofe)\b',
        r'\b(asesinato|masacre|genocidio|crimen atroz|violación grave)\b',
        r'\b(colapso|caos|devastación|pánico|terror)\b',
        r'\b(repudio|indignación|brutal|inhumano|inaceptable)\b' # <-- AÑADIDO
    ],
    "moderate_negative": [
        r'\b(preocupación|problema|denuncia|crítica|polémica)\b',
        r'\b(deterioro|déficit|irregularidad|falla|deficiencia)\b',
        r'\b(protestas|conflicto|rechazo|oposición|controversia)\b',
        r'\b(amenaza|riesgo|peligro|alerta|advertencia)\b',
        r'\b(grave|difícil|complicada|lamentable|impacto negativo)\b', # <-- AÑADIDO
        r'\b(incertidumbre|retroceso|afecta|perjudica)\b' # <-- AÑADIDO
    ],
    "strong_positive": [
        r'\b(logro histórico|éxito rotundo|hito|hazaña|triunfo)\b',
        r'\b(reconocimiento internacional|premio prestigioso|distinción)\b',
        r'\b(récord|breakthrough|avance revolucionario)\b',
        r'\b(excelente|magnífico|extraordinario|orgullo nacional)\b' # <-- AÑADIDO
    ],
    "moderate_positive": [
        r'\b(mejora|avance|progreso|lanzamiento|premio|auge|crecimiento|desarrollo)\b',
        r'\b(inversión|acuerdo|aprobación|beneficio|ganancia|lanza|soluciones|)\b',
        r'\b(inauguración|entrega|lanzamiento|apertura)\b',
        r'\b(victoria|éxito|logro|conquista)\b',
        r'\b(impulso|fortalece|recuperación|oportunidad|positivo)\b' # <-- AÑADIDO
    ]
}

# Temas sensibles que requieren análisis cuidadoso
SENSITIVE_TOPICS = {
    "Seguridad y Justicia": ["Negativo", "Neutro"],  # Raramente positivo
    "Social y Derechos Humanos": ["Negativo", "Neutro"],
    "Medio Ambiente y Clima": ["Negativo", "Neutro"]
}

TOPIC_LIST = list(TOPIC_KEYWORDS.keys())

# ===============================================================
# UTILIDADES MEJORADAS
# ===============================================================

def sanitize_name(name):
    return re.sub(r"[^A-Za-z0-9_\-]", "_", name)

def clean_text_enhanced(text):
    """Limpieza de texto mejorada con preservación de contexto"""
    if not isinstance(text, str):
        return ""

    t = text.strip()

    # Preservar puntuación importante para contexto
    t = re.sub(r'https?://\S+', ' [URL] ', t)
    t = re.sub(r'[\x00-\x1f\x7f]', ' ', t)
    t = re.sub(r'\s+', ' ', t)

    # Truncar inteligentemente
    if len(t) > MAX_CHARS_CLEAN:
        truncated = t[:MAX_CHARS_CLEAN]
        # Buscar punto final o punto y coma cercano
        last_period = max(truncated.rfind('.'), truncated.rfind(';'),
                          truncated.rfind('!'), truncated.rfind('?'))

        if last_period > MAX_CHARS_CLEAN * 0.7:
            t = truncated[:last_period + 1]
        else:
            t = truncated.rsplit(' ', 1)[0] if ' ' in truncated else truncated

    return t.strip()

def normalize_for_comparison(text):
    """Normalización avanzada para comparación"""
    t = text.lower().strip()

    # Remover URLs y menciones
    t = re.sub(r'https?://\S+|www\.\S+|@\w+', ' ', t)

    # Normalizar números (mantener significado pero simplificar)
    t = re.sub(r'\d+', 'NUM', t)

    # Remover puntuación pero mantener estructura
    t = re.sub(r'[^\w\sáéíóúüñ]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()

    return t

def get_text_signature(text, n_words=PREFIX_WORDS):
    """Obtiene firma del texto para comparación rápida"""
    words = normalize_for_comparison(text).split()[:n_words]
    return ' '.join(words)

def analyze_sentiment_patterns(text):
    """Análisis de patrones de sentimiento en el texto"""
    text_lower = text.lower()

    scores = {
        'strong_negative': 0,
        'moderate_negative': 0,
        'strong_positive': 0,
        'moderate_positive': 0
    }

    for sentiment_type, patterns in SENTIMENT_PATTERNS.items():
        for pattern in patterns:
            matches = len(re.findall(pattern, text_lower))
            scores[sentiment_type] += matches

    return scores

def calculate_sentiment_score(pattern_scores):
    """Calcula score de sentimiento basado en patrones"""
    neg_score = (pattern_scores['strong_negative'] * 2 +
                 pattern_scores['moderate_negative'])
    pos_score = (pattern_scores['strong_positive'] * 2 +
                 pattern_scores['moderate_positive'])

    # --- AJUSTE DE PRECISIÓN (Menos Neutro) ---
    # Se reduce el umbral de +2 a +0 para ser más sensible
    if neg_score > pos_score:
        return 'Negativo'
    elif pos_score > neg_score:
        return 'Positivo'
    else:
        return 'Neutro'

def print_gpu_usage():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        print(f"   GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")

# ===============================================================
# AGRUPAMIENTO INTELIGENTE MEJORADO
# ===============================================================

def agrupar_textos_avanzado(textos):
    """
    Agrupamiento multi-nivel:
    1. Pre-filtro por firma textual
    2. Clustering por similitud semántica
    3. Post-procesamiento de micro-clusters
    """
    print("🔍 Agrupamiento inteligente multi-nivel...")

    # NIVEL 1: Agrupamiento por firma textual
    print("   Nivel 1: Agrupamiento por firma textual")
    signature_groups = defaultdict(list)

    for i, txt in enumerate(textos):
        if len(txt.strip()) < 30:
            sig = f"_short_{i}"
        else:
            sig = get_text_signature(txt)
        signature_groups[sig].append((i, txt))

    print(f"   → {len(signature_groups)} grupos de firma")

    # NIVEL 2: Clustering semántico dentro de cada grupo de firma
    print("   Nivel 2: Clustering semántico")
    encoder = SentenceTransformer(MODELO_EMBEDDING, device=device)
    encoder.max_seq_length = 256

    final_clusters = []
    cluster_map = np.zeros(len(textos), dtype=np.int32)
    current_cluster_id = 0

    for sig, items in tqdm(signature_groups.items(), desc="Procesando firmas"):
        if len(items) == 1:
            idx, txt = items[0]
            final_clusters.append({
                'indices': [idx],
                'representative': txt,
                'size': 1
            })
            cluster_map[idx] = current_cluster_id
            current_cluster_id += 1
            continue

        # Obtener embeddings para el grupo
        indices, texts = zip(*items)
        embeddings = encoder.encode(
            texts,
            batch_size=BATCH_EMB,
            show_progress_bar=False,
            convert_to_tensor=True,
            normalize_embeddings=True,
            device=device
        )

        # Clustering jerárquico para mejor control
        if len(items) <= 5:
            # Grupos pequeños: usar threshold estricto
            clustering = AgglomerativeClustering(
                n_clusters=None,
                distance_threshold=1 - SIMILARITY_THRESHOLD_STRICT,
                linkage='average',
                metric='cosine'
            )
        else:
            # Grupos grandes: DBSCAN para identificar outliers
            clustering = DBSCAN(
                eps=1 - SIMILARITY_THRESHOLD_MODERATE,
                min_samples=MIN_CLUSTER_SIZE,
                metric='cosine'
            )

        labels = clustering.fit_predict(embeddings.cpu().numpy())

        # Crear sub-clusters
        label_groups = defaultdict(list)
        for label, idx, txt in zip(labels, indices, texts):
            label_groups[label].append((idx, txt))

        # Asignar clusters finales
        for label, sub_items in label_groups.items():
            sub_indices, sub_texts = zip(*sub_items)

            # Seleccionar representante (el más largo y centrado)
            if len(sub_items) > 1:
                # Calcular texto más centrado semántica
                sub_embs_indices = [indices.index(idx) for idx in sub_indices]
                sub_embs = embeddings[torch.tensor(sub_embs_indices).to(device)]
                centroid = sub_embs.mean(dim=0)
                distances = torch.cosine_similarity(sub_embs, centroid.unsqueeze(0))
                rep_idx = sub_indices[distances.argmax().item()]
                rep_text = textos[rep_idx]
            else:
                rep_idx = sub_indices[0]
                rep_text = sub_texts[0]

            final_clusters.append({
                'indices': list(sub_indices),
                'representative': rep_text,
                'representative_idx': rep_idx,
                'size': len(sub_indices)
            })

            for idx in sub_indices:
                cluster_map[idx] = current_cluster_id

            current_cluster_id += 1

    # NIVEL 3: Post-procesamiento - fusionar micro-clusters muy similares
    print("   Nivel 3: Fusión de micro-clusters")

    if len(final_clusters) > 100:  # Solo si hay muchos clusters
        rep_texts = [c['representative'] for c in final_clusters]
        rep_embeddings = encoder.encode(
            rep_texts,
            batch_size=BATCH_EMB,
            show_progress_bar=False,
            convert_to_tensor=True,
            normalize_embeddings=True,
            device=device
        )

        # Identificar pares muy similares
        similarity_matrix = util.cos_sim(rep_embeddings, rep_embeddings)
        merged = set()

        for i in range(len(final_clusters)):
            if i in merged:
                continue

            for j in range(i + 1, len(final_clusters)):
                if j in merged:
                    continue

                if similarity_matrix[i][j] > SIMILARITY_THRESHOLD_STRICT:
                    # Fusionar cluster j en cluster i
                    final_clusters[i]['indices'].extend(final_clusters[j]['indices'])
                    final_clusters[i]['size'] += final_clusters[j]['size']

                    # Actualizar mapa (apuntar todos los textos de j al ID de i)
                    target_cluster_id = cluster_map[final_clusters[i]['indices'][0]]
                    for idx in final_clusters[j]['indices']:
                        cluster_map[idx] = target_cluster_id

                    merged.add(j)

        # Remover clusters fusionados
        final_clusters = [c for i, c in enumerate(final_clusters) if i not in merged]
        print(f"   → Fusionados {len(merged)} micro-clusters")

        # --- INICIO DE LA CORRECCIÓN DEL INDEXERROR ---
        # Es crucial reconstruir el cluster_map DESPUÉS de fusionar y
        # re-indexar la lista 'final_clusters'.
        # El 'cluster_map' anterior contenía IDs (ej. 150) que ahora están
        # fuera de rango si la lista 'final_clusters' se acortó (ej. a 149).
        print(f"   Nivel 3b: Re-indexando cluster_map final...")
        cluster_map_final = np.zeros(len(textos), dtype=np.int32)
        for new_cluster_id, cluster_info in enumerate(final_clusters):
            for idx in cluster_info['indices']:
                cluster_map_final[idx] = new_cluster_id

        # Sobrescribir el cluster_map antiguo con el mapa re-indexado
        cluster_map = cluster_map_final
        # --- FIN DE LA CORRECCIÓN ---

    representatives = [c['representative'] for c in final_clusters]

    reduccion = (1 - len(representatives)/max(1, len(textos))) * 100
    print(f"✓ {len(textos)} textos → {len(representatives)} clusters ({reduccion:.1f}% reducción)")
    print(f"   Distribución de tamaños: "
          f"1 texto: {sum(1 for c in final_clusters if c['size'] == 1)}, "
          f"2-5 textos: {sum(1 for c in final_clusters if 2 <= c['size'] <= 5)}, "
          f">5 textos: {sum(1 for c in final_clusters if c['size'] > 5)}")

    del encoder, embeddings, rep_embeddings
    gc.collect()
    torch.cuda.empty_cache()

    return representatives, cluster_map, final_clusters

# ===============================================================
# ANÁLISIS DE SENTIMIENTO MEJORADO
# ===============================================================

def inferir_sentimiento_avanzado(representatives, textos_completos, cluster_map, clusters):
    """
    Análisis de sentimiento con sistema de votación para clusters
    """
    print(f"\n🎯 Análisis de sentimiento avanzado con {MODELO_SENTIMIENTO}")
    print(f"   Batch size: {BATCH_SENT}")

    t_start = time.time()

    # Análisis de sentimiento base
    sentiment_pipe = pipeline(
        "text-classification",
        model=MODELO_SENTIMIENTO,
        device=0 if torch.cuda.is_available() else -1,
        batch_size=BATCH_SENT,
        truncation=True,
        max_length=MAX_LEN_SENT
    )

    print_gpu_usage()

    # Procesar representantes
    resultados_rep = []
    # Manejar caso de 0 representantes (ej. 0 textos de entrada)
    if len(representatives) > 0:
        for i in tqdm(range(0, len(representatives), BATCH_SENT), desc="Sentimiento base"):
            batch = representatives[i:i + BATCH_SENT]
            outputs = sentiment_pipe(batch)
            resultados_rep.extend(outputs)

    label_map = {
        "Positive": "Positivo",
        "Negative": "Negativo",
        "Neutral": "Neutro"
    }

    # Sentimientos base de representantes
    sentimientos_base = [label_map.get(r['label'], 'Neutro') for r in resultados_rep]

    # REFINAMIENTO: Análisis de patrones para clusters grandes
    print("   Refinando sentimiento en clusters grandes...")
    sentimientos_refinados = []

    for cluster_id, cluster_info in enumerate(tqdm(clusters, desc="Refinamiento")):
        if cluster_info['size'] == 1:
            # Cluster unitario: usar análisis de patrones
            idx = cluster_info['indices'][0]
            texto = textos_completos[idx]
            pattern_scores = analyze_sentiment_patterns(texto)
            pattern_sentiment = calculate_sentiment_score(pattern_scores)

            base_sent = sentimientos_base[cluster_id]

            # --- AJUSTE DE PRECISIÓN (Menos Neutro) ---
            # Priorizar el sentimiento de patrones (que ahora es más sensible)
            # si este NO es Neutro.
            if pattern_sentiment != 'Neutro':
                final_sent = pattern_sentiment
            else:
                final_sent = base_sent # Usar el base solo si patrones no detectan nada

            sentimientos_refinados.append(final_sent)

        else:
            # Cluster múltiple: sistema de votación
            indices = cluster_info['indices']
            votos = []

            for idx in indices[:min(5, len(indices))]:  # Máximo 5 muestras por cluster
                texto = textos_completos[idx]
                pattern_scores = analyze_sentiment_patterns(texto)
                pattern_sent = calculate_sentiment_score(pattern_scores)
                votos.append(pattern_sent)

            # Agregar voto del análisis base
            votos.append(sentimientos_base[cluster_id])

            # Votación ponderada
            voto_counts = Counter(votos)
            sentimiento_final = voto_counts.most_common(1)[0][0]

            # Si hay empate, usar el análisis base
            if len(voto_counts) > 1 and list(voto_counts.values())[0] == list(voto_counts.values())[1]:
                sentimiento_final = sentimientos_base[cluster_id]

            sentimientos_refinados.append(sentimiento_final)

    t_elapsed = time.time() - t_start
    velocidad = len(representatives) / max(1, t_elapsed)
    print(f"   ⚡ Procesados {len(representatives)} clusters en {t_elapsed:.2f}s ({velocidad:.0f} clusters/s)")
    print_gpu_usage()

    del sentiment_pipe
    gc.collect()
    torch.cuda.empty_cache()

    return sentimientos_refinados

# ===============================================================
# CLASIFICACIÓN DE TEMAS MEJORADA
# ===============================================================

def inferir_temas_avanzado(representatives):
    """
    Clasificación de temas con análisis jerárquico:
    1. Matching de keywords primary (alta confianza)
    2. Embeddings semánticos
    3. Refinamiento con keywords secondary
    """
    print(f"\n🏷️ Clasificación de temas avanzada con {MODELO_EMBEDDING}")
    print(f"   Batch size: {BATCH_EMB}")

    # Manejar caso de 0 representantes
    if len(representatives) == 0:
        print("   → No hay representantes para analizar.")
        return []

    t_start = time.time()

    encoder = SentenceTransformer(MODELO_EMBEDDING, device=device)
    encoder.max_seq_length = 256

    print_gpu_usage()

    # PASO 1: Análisis directo de keywords primary
    print("   Paso 1: Análisis de keywords primarias")
    keyword_matches = []

    for txt in representatives:
        txt_lower = txt.lower()
        topic_scores = {}

        for topic, keywords_dict in TOPIC_KEYWORDS.items():
            primary_score = sum(2 for kw in keywords_dict['primary'] if kw in txt_lower)
            secondary_score = sum(1 for kw in keywords_dict['secondary'] if kw in txt_lower)
            topic_scores[topic] = primary_score + secondary_score

        # Si hay match fuerte con keywords primary (score >= 2), usar ese tema
        max_score = max(topic_scores.values()) if topic_scores else 0
        if max_score >= 2:
            best_topic = max(topic_scores, key=topic_scores.get)
            keyword_matches.append((best_topic, max_score))
        else:
            keyword_matches.append((None, max_score))

    n_keyword_matches = sum(1 for match, _ in keyword_matches if match is not None)
    print(f"   → {n_keyword_matches}/{len(representatives)} con match directo")

    # PASO 2: Embeddings semánticos para textos sin match claro
    print("   Paso 2: Análisis semántico con embeddings")

    # Crear centroides mejorados
    centroides_frases = []
    for tema, keywords_dict in TOPIC_KEYWORDS.items():
        # Combinar primary y secondary para mejor representación
        all_keywords = keywords_dict['primary'][:8] + keywords_dict['secondary'][:4]
        frase = f"{tema}: " + ", ".join(all_keywords)
        centroides_frases.append(frase)

    centroides = encoder.encode(
        centroides_frases,
        batch_size=32,
        show_progress_bar=False,
        convert_to_tensor=True,
        normalize_embeddings=True,
        device=device
    )

    # Encodear representantes
    all_vecs = []
    for i in tqdm(range(0, len(representatives), BATCH_EMB), desc="Embeddings"):
        batch = representatives[i:i + BATCH_EMB]
        vecs = encoder.encode(
            batch,
            batch_size=BATCH_EMB,
            show_progress_bar=False,
            convert_to_tensor=True,
            normalize_embeddings=True,
            device=device
        )
        all_vecs.append(vecs)

    all_vecs = torch.cat(all_vecs, dim=0)
    similarities = torch.matmul(all_vecs, centroides.T)
    tema_ids_embeddings = similarities.argmax(dim=1).cpu().numpy()
    max_similarities = similarities.max(dim=1).values.cpu().numpy()

    # PASO 3: Decisión final combinando ambos métodos
    print("   Paso 3: Combinación y refinamiento")
    temas_finales = []

    for i, (keyword_match, keyword_score) in enumerate(keyword_matches):
        if keyword_match is not None and keyword_score >= 4:
            # Match muy fuerte de keywords -> usar ese tema
            temas_finales.append(keyword_match)
        elif keyword_match is not None and max_similarities[i] < 0.3:
            # Match moderado de keywords pero baja similitud semántica
            # Confiar en keywords
            temas_finales.append(keyword_match)
        elif max_similarities[i] >= 0.4:
            # Alta similitud semántica -> usar embedding
            temas_finales.append(TOPIC_LIST[int(tema_ids_embeddings[i])])
        elif keyword_match is not None:
            # Match débil de keywords -> usar ese tema
            temas_finales.append(keyword_match)
        else:
            # Usar embeddings como fallback
            temas_finales.append(TOPIC_LIST[int(tema_ids_embeddings[i])])

    t_elapsed = time.time() - t_start
    velocidad = len(representatives) / max(1, t_elapsed)
    print(f"   ⚡ Procesados {len(representatives)} textos en {t_elapsed:.2f}s ({velocidad:.0f} textos/s)")
    print_gpu_usage()

    del encoder, centroides, all_vecs, similarities
    gc.collect()
    torch.cuda.empty_cache()

    return temas_finales

# ===============================================================
# POST-PROCESAMIENTO DE CONSISTENCIA
# ===============================================================

def post_procesar_consistencia(temas, sentimientos, clusters, textos_originales):
    """
    Asegura consistencia tema-tono basada en contexto colombiano
    """
    print("\n🔧 Post-procesamiento de consistencia tema-tono...")

    ajustes = 0

    for cluster_id, (tema, sentimiento) in enumerate(zip(temas, sentimientos)):
        # Verificar temas sensibles
        if tema in SENSITIVE_TOPICS:
            expected_sentiments = SENSITIVE_TOPICS[tema]

            if sentimiento not in expected_sentiments:
                # Revisar el contenido del cluster
                cluster_indices = clusters[cluster_id]['indices']

                # Analizar algunos textos del cluster
                sample_size = min(3, len(cluster_indices))
                negative_indicators = 0

                for idx in cluster_indices[:sample_size]:
                    texto = textos_originales[idx].lower()

                    # Buscar indicadores negativos fuertes
                    if any(word in texto for word in [
                        'asesinato', 'masacre', 'muerto', 'víctima', 'violencia',
                        'amenaza', 'ataque', 'denuncia', 'crisis', 'desastre'
                    ]):
                        negative_indicators += 1

                # Si hay evidencia fuerte de negatividad, ajustar
                if negative_indicators >= sample_size * 0.6:
                    if sentimiento == "Positivo":
                        sentimientos[cluster_id] = "Neutro"
                        ajustes += 1

        # Verificar inconsistencias tema-sentimiento específicas
        if tema == "Deportes y Entretenimiento" and sentimiento == "Negativo":
            # Deportes raramente es muy negativo, revisar
            cluster_indices = clusters[cluster_id]['indices']
            texto_muestra = textos_originales[cluster_indices[0]].lower()

            # Solo mantener negativo si hay palabras muy negativas
            if not any(word in texto_muestra for word in [
                'muerte', 'lesión grave', 'escándalo', 'dopaje', 'corrupción', 'tragedia'
            ]):
                sentimientos[cluster_id] = "Neutro"
                ajustes += 1

        # Economía con keywords positivos pero clasificado como negativo
        if tema == "Economía y Finanzas" and sentimiento == "Negativo":
            cluster_indices = clusters[cluster_id]['indices']
            texto_muestra = textos_originales[cluster_indices[0]].lower()

            positive_econ_words = [
                'crecimiento', 'aumento exportaciones', 'inversión récord',
                'mejora pib', 'reducción desempleo', 'fortalecimiento'
            ]

            if any(word in texto_muestra for word in positive_econ_words):
                sentimientos[cluster_id] = "Neutro"
                ajustes += 1

    if ajustes > 0:
        print(f"   ✓ Realizados {ajustes} ajustes de consistencia")
    else:
        print(f"   ✓ Sin ajustes necesarios - alta consistencia")

    return temas, sentimientos

# ===============================================================
# ANÁLISIS DE CALIDAD
# ===============================================================

def analizar_calidad_resultados(df, clusters):
    """
    Analiza la calidad y consistencia de los resultados
    """
    print("\n📊 Análisis de calidad:")

    # Manejar caso de 0 clusters
    if len(clusters) == 0:
        print("   • No hay clusters para analizar.")
        return

    # Consistencia intra-cluster
    inconsistencias_tono = 0
    inconsistencias_tema = 0

    total_clusters_multi = sum(1 for c in clusters if c['size'] > 1)

    if total_clusters_multi > 0:
        for cluster in clusters:
            if cluster['size'] > 1:
                indices = cluster['indices']
                tonos = df.iloc[indices]['tono'].values
                temas = df.iloc[indices]['tema'].values

                # Si hay variación en un cluster, es inconsistencia
                if len(set(tonos)) > 1:
                    inconsistencias_tono += 1
                if len(set(temas)) > 1:
                    inconsistencias_tema += 1

        consistencia_tono = (1 - inconsistencias_tono / total_clusters_multi) * 100
        consistencia_tema = (1 - inconsistencias_tema / total_clusters_multi) * 100

        print(f"   • Consistencia de tono en clusters: {consistencia_tono:.1f}%")
        print(f"   • Consistencia de tema en clusters: {consistencia_tema:.1f}%")
    else:
        print("   • No hay clusters de tamaño > 1 para analizar consistencia.")

    # Distribución de confianza
    distribucion_cluster_sizes = Counter(c['size'] for c in clusters)
    print(f"\n   • Distribución de tamaños de cluster:")
    for size in sorted(distribucion_cluster_sizes.keys())[:5]:
        count = distribucion_cluster_sizes[size]
        print(f"       - {size} textos: {count} clusters")

    # Coverage de temas
    temas_unicos = df['tema'].nunique() if not df.empty else 0
    print(f"\n   • Temas identificados: {temas_unicos}/{len(TOPIC_LIST)}")

    # Balance de sentimientos
    if not df.empty:
        balance_sent = df['tono'].value_counts(normalize=True) * 100
        print(f"\n   • Balance de sentimientos:")
        for sent, pct in balance_sent.items():
            print(f"       - {sent}: {pct:.1f}%")
    else:
        print("\n   • Balance de sentimientos: No hay datos.")


# ===============================================================
# EXPORTACIÓN MEJORADA
# ===============================================================

def guardar_resultados_avanzado(df, nombre_original, clusters, t_total, metricas_calidad=None):
    """
    Guarda resultados con análisis extendido y métricas de calidad
    """
    # --- CORRECCIÓN DEL ERROR DE SINTAXIS ---
    # La línea original estaba dividida incorrectamente.
    # Se ha unido en una sola línea y se ha añadido '$' para asegurar
    # que solo elimina la extensión al final del nombre.
    base = re.sub(r'\.xlsx?$', '', nombre_original, flags=re.IGNORECASE)
    # --- FIN DE LA CORRECCIÓN ---

    ts = time.strftime("%Y%m%d_%H%M%S")
    out_xlsx = f"/content/analisis_avanzado_v15_{sanitize_name(base)}_{ts}.xlsx"

    n = len(df)
    n_clusters = len(clusters)

    print("\n📝 Generando Excel con análisis completo...")

    # Estadísticas básicas
    dist_tono = df['tono'].value_counts() if not df.empty else pd.Series()
    dist_tema = df['tema'].value_counts() if not df.empty else pd.Series()
    crosstab = pd.crosstab(df['tema'], df['tono'], margins=True) if not df.empty else pd.DataFrame()

    # Análisis de clusters
    cluster_sizes = [c['size'] for c in clusters]
    cluster_stats = pd.DataFrame({
        'Tamaño Cluster': sorted(set(cluster_sizes)),
        'Cantidad': [cluster_sizes.count(s) for s in sorted(set(cluster_sizes))],
        'Total Textos': [cluster_sizes.count(s) * s for s in sorted(set(cluster_sizes))]
    }) if cluster_sizes else pd.DataFrame()

    # Top temas por sentimiento
    tema_tono_detailed = []
    if not df.empty:
        for tema in TOPIC_LIST:
            tema_df = df[df['tema'] == tema]
            if len(tema_df) > 0:
                for tono in ['Positivo', 'Neutro', 'Negativo']:
                    count = len(tema_df[tema_df['tono'] == tono])
                    pct = (count / len(tema_df)) * 100 if len(tema_df) > 0 else 0
                    tema_tono_detailed.append({
                        'Tema': tema,
                        'Tono': tono,
                        'Cantidad': count,
                        'Porcentaje': round(pct, 2)
                    })

    tema_tono_df = pd.DataFrame(tema_tono_detailed)

    with pd.ExcelWriter(out_xlsx, engine="openpyxl") as w:
        # Hoja 1: Resultados principales
        if not df.empty:
            cols_order = ["resumen", "tono", "tema", "cluster_id", "cluster_size"] + \
                         [c for c in df.columns if c not in ["resumen", "tono", "tema", "cluster_id", "cluster_size"]]
            df[cols_order].to_excel(w, sheet_name="Resultados", index=False)
        else:
            pd.DataFrame([{"Error": "No se procesaron datos"}]).to_excel(w, sheet_name="Resultados", index=False)

        # Hoja 2: Estadísticas generales
        stats = pd.DataFrame({
            "Métrica": [
                "Total de textos",
                "Clusters procesados",
                "Reducción",
                "Textos únicos",
                "Tiempo total (s)",
                "Velocidad (textos/s)",
                "Batch Sentimiento",
                "Batch Embeddings",
                "Modelo Sentimiento",
                "Modelo Temas",
                "Similarity Threshold Strict",
                "Similarity Threshold Moderate"
            ],
            "Valor": [
                n,
                n_clusters,
                f"{(1 - n_clusters/max(n,1))*100:.1f}%",
                sum(1 for c in clusters if c['size'] == 1),
                f"{t_total:.2f}",
                f"{n/max(t_total,1):.0f}",
                BATCH_SENT,
                BATCH_EMB,
                MODELO_SENTIMIENTO,
                MODELO_EMBEDDING,
                SIMILARITY_THRESHOLD_STRICT,
                SIMILARITY_THRESHOLD_MODERATE
            ],
        })
        stats.to_excel(w, sheet_name="Estadisticas", index=False)

        # Hoja 3: Distribución de tonos
        if not dist_tono.empty:
            dist_tono_df = pd.DataFrame({
                'Tono': dist_tono.index,
                'Cantidad': dist_tono.values,
                'Porcentaje': (dist_tono.values / max(1, n) * 100).round(2)
            })
            dist_tono_df.to_excel(w, sheet_name="Distribucion_Tonos", index=False)

        # Hoja 4: Distribución de temas
        if not dist_tema.empty:
            dist_tema_df = pd.DataFrame({
                'Tema': dist_tema.index,
                'Cantidad': dist_tema.values,
                'Porcentaje': (dist_tema.values / max(1, n) * 100).round(2)
            }).sort_values('Cantidad', ascending=False)
            dist_tema_df.to_excel(w, sheet_name="Distribucion_Temas", index=False)

        # Hoja 5: Cruce tema-tono
        if not crosstab.empty:
            crosstab.to_excel(w, sheet_name="Cruce_Tema_Tono")

        # Hoja 6: Análisis detallado tema-tono
        if not tema_tono_df.empty:
            tema_tono_df.to_excel(w, sheet_name="Detalle_Tema_Tono", index=False)

        # Hoja 7: Análisis de clusters
        if not cluster_stats.empty:
            cluster_stats.to_excel(w, sheet_name="Analisis_Clusters", index=False)

        # Hoja 8: Ejemplos por tema
        ejemplos_tema = []
        if not dist_tema.empty:
            for tema in dist_tema.index[:10]:  # Top 10 temas
                tema_samples = df[df['tema'] == tema].head(3)
                for _, row in tema_samples.iterrows():
                    ejemplos_tema.append({
                        'Tema': tema,
                        'Tono': row['tono'],
                        'Resumen': str(row['resumen'])[:200] + '...' if len(str(row['resumen'])) > 200 else row['resumen']
                    })

        if ejemplos_tema:
            pd.DataFrame(ejemplos_tema).to_excel(w, sheet_name="Ejemplos_Temas", index=False)

    if os.path.exists(out_xlsx):
        print(f"💾 Descargando: {os.path.basename(out_xlsx)}")
        try:
            files.download(out_xlsx)
            print(f"✅ Descarga completada")
        except Exception as e:
            print(f"⚠️ Error al descargar: {e}")
            print(f"📁 Archivo disponible en: {out_xlsx}")

# ===============================================================
# FLUJO PRINCIPAL
# ===============================================================

def main():
    print("="*70)
    print("   ANALIZADOR AVANZADO DE NOTICIAS COLOMBIA v15")
    print("   Multi-nivel Clustering + Votación + Post-procesamiento")
    print("="*70)
    print(f"⚡ Batch Sentimiento: {BATCH_SENT}")
    print(f"⚡ Batch Embeddings: {BATCH_EMB}")
    print(f"🔬 Modelo Sentimiento: {MODELO_SENTIMIENTO}")
    print(f"🔬 Modelo Temas: {MODELO_EMBEDDING}")
    print(f"📊 Temas configurados: {len(TOPIC_LIST)}")
    print(f"🎯 Similarity Strict: {SIMILARITY_THRESHOLD_STRICT}")
    print(f"🎯 Similarity Moderate: {SIMILARITY_THRESHOLD_MODERATE}")
    print("="*70)

    try:
        print("\n📤 Suba su archivo Excel con columna 'resumen'")
        up = files.upload()
        if not up:
            print("❌ No se subió archivo")
            return

        nombre = list(up.keys())[0]
        print(f"\n📊 Leyendo: {nombre}")
        df = pd.read_excel(io.BytesIO(up[nombre]))

    except Exception as e:
        print(f"❌ Error fatal al cargar el archivo: {e}")
        return

    if 'resumen' not in df.columns:
        print(f"❌ Error: Falta columna 'resumen'. Columnas: {', '.join(df.columns)}")
        return

    # Manejar DataFrames vacíos
    if df.empty:
        print("⚠️ El archivo está vacío o no tiene filas.")
        # Aún así, intentamos guardar un reporte vacío
        df['tono'] = []
        df['tema'] = []
        df['cluster_id'] = []
        df['cluster_size'] = []
        t0 = time.time()
        clusters = []
    else:
        print(f"✓ {len(df)} textos cargados")

        # Preparar textos
        textos_originales = df['resumen'].fillna('').astype(str).tolist()
        textos_limpios = [clean_text_enhanced(t) for t in textos_originales]

        print("\n" + "="*70)
        print("INICIANDO PROCESAMIENTO AVANZADO")
        print("="*70)

        t0 = time.time()

        # FASE 1: Agrupamiento multi-nivel
        representatives, cluster_map, clusters = agrupar_textos_avanzado(textos_limpios)

        # FASE 2: Análisis de sentimiento con votación
        tonos_clusters = inferir_sentimiento_avanzado(
            representatives,
            textos_originales,
            cluster_map,
            clusters
        )

        # FASE 3: Clasificación de temas avanzada
        temas_clusters = inferir_temas_avanzado(representatives)

        # FASE 4: Post-procesamiento de consistencia
        temas_clusters, tonos_clusters = post_procesar_consistencia(
            temas_clusters,
            tonos_clusters,
            clusters,
            textos_originales
        )

        # Mapear resultados a todos los textos
        n = len(df)
        # --- Verificación para prevenir el error si las listas están vacías ---
        if n > 0 and len(tonos_clusters) > 0 and len(temas_clusters) > 0:
            df['tono'] = [tonos_clusters[int(cluster_map[i])] for i in range(n)]
            df['tema'] = [temas_clusters[int(cluster_map[i])] for i in range(n)]
            df['cluster_id'] = cluster_map
            df['cluster_size'] = [clusters[int(cluster_map[i])]['size'] for i in range(n)]
        elif n > 0:
             print("⚠️ Error: No se generaron clusters pero hay datos. Asignando N/A.")
             df['tono'] = 'N/A'
             df['tema'] = 'N/A'
             df['cluster_id'] = -1
             df['cluster_size'] = 0
        else:
             # df está vacío, crear columnas vacías
             df['tono'] = []
             df['tema'] = []
             df['cluster_id'] = []
             df['cluster_size'] = []


    t1 = time.time()
    n = len(df) # Recalcular n por si estaba vacío
    t_total = t1 - t0

    print(f"\n{'='*70}")
    print(f"✅ PROCESAMIENTO COMPLETADO")
    print(f"⚡ Tiempo total: {t_total:.2f} segundos")
    if t_total > 0:
        print(f"⚡ Velocidad: {n/t_total:.0f} textos/segundo")
    print(f"{'='*70}")

    # Análisis de calidad
    analizar_calidad_resultados(df, clusters)

    print("\n📊 DISTRIBUCIÓN FINAL:")
    if not df.empty:
        print(f"\nTonos:\n{df['tono'].value_counts()}")
        print(f"\nTop 5 Temas:\n{df['tema'].value_counts().head(5)}")
    else:
        print("No hay datos para mostrar distribución.")


    print(f"\n🎯 MÉTRICAS DE OPTIMIZACIÓN:")
    print(f"   • Textos procesados: {n}")
    print(f"   • Clusters creados: {len(clusters)}")
    if n > 0:
        print(f"   • Eficiencia: {(1 - len(clusters)/n)*100:.1f}% reducción")
    if clusters:
        print(f"   • Clusters únicos: {sum(1 for c in clusters if c['size'] == 1)}")
        print(f"   • Clusters múltiples: {sum(1 for c in clusters if c['size'] > 1)}")
        print(f"   • Cluster más grande: {max(c['size'] for c in clusters)} textos")

    # Guardar resultados
    guardar_resultados_avanzado(df, nombre, clusters, t_total)

    # Limpieza final
    gc.collect()
    torch.cuda.empty_cache()

    print("\n✅ Proceso completado exitosamente")
    print("="*70)

if __name__ == '__main__':
    main()

✓ GPU: Tesla T4
✓ VRAM Total: 14.7 GB
   ANALIZADOR AVANZADO DE NOTICIAS COLOMBIA v15
   Multi-nivel Clustering + Votación + Post-procesamiento
⚡ Batch Sentimiento: 64
⚡ Batch Embeddings: 256
🔬 Modelo Sentimiento: clapAI/roberta-large-multilingual-sentiment
🔬 Modelo Temas: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
📊 Temas configurados: 10
🎯 Similarity Strict: 0.92
🎯 Similarity Moderate: 0.85

📤 Suba su archivo Excel con columna 'resumen'


Saving resumen_concatenado (1).xlsx to resumen_concatenado (1) (5).xlsx

📊 Leyendo: resumen_concatenado (1) (5).xlsx
✓ 262 textos cargados

INICIANDO PROCESAMIENTO AVANZADO
🔍 Agrupamiento inteligente multi-nivel...
   Nivel 1: Agrupamiento por firma textual
   → 214 grupos de firma
   Nivel 2: Clustering semántico


Procesando firmas:   0%|          | 0/214 [00:00<?, ?it/s]

   Nivel 3: Fusión de micro-clusters
   → Fusionados 39 micro-clusters
   Nivel 3b: Re-indexando cluster_map final...
✓ 262 textos → 179 clusters (31.7% reducción)
   Distribución de tamaños: 1 texto: 129, 2-5 textos: 46, >5 textos: 4

🎯 Análisis de sentimiento avanzado con clapAI/roberta-large-multilingual-sentiment
   Batch size: 64


Device set to use cuda:0


...

Embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

   Paso 3: Combinación y refinamiento
   ⚡ Procesados 179 textos en 4.86s (37 textos/s)
   GPU Memory: 1.05 GB allocated, 2.61 GB reserved

🔧 Post-procesamiento de consistencia tema-tono...
   ✓ Realizados 3 ajustes de consistencia

✅ PROCESAMIENTO COMPLETADO
⚡ Tiempo total: 15.78 segundos
⚡ Velocidad: 17 textos/segundo

📊 Análisis de calidad:
   • Consistencia de tono en clusters: 100.0%
   • Consistencia de tema en clusters: 100.0%

   • Distribución de tamaños de cluster:
       - 1 textos: 129 clusters
       - 2 textos: 38 clusters
       - 3 textos: 6 clusters
       - 4 textos: 2 clusters
       - 6 textos: 1 clusters

   • Temas identificados: 10/10

   • Balance de sentimientos:
       - Neutro: 67.9%
       - Positivo: 17.6%
       - Negativo: 14.5%

📊 DISTRIBUCIÓN FINAL:

Tonos:
tono
Neutro      178
Positivo     46
Negativo     38
Name: count, dtype: int64

Top 5 Temas:
tema
Regiones y Territorio           83
Salud y Bienestar               82
Política y Gobierno            

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga completada

✅ Proceso completado exitosamente


#Detalles

### Explicación Detallada del Código: Analizador de Tono y Tema v15

Este código Python es una herramienta robusta para el análisis de tono y tema en textos, optimizada para el contexto colombiano. Utiliza técnicas avanzadas de Procesamiento del Lenguaje Natural (PLN) y Machine Learning para ofrecer resultados precisos y organizados.

Aquí desglosamos sus componentes y funcionamiento:

1.  **Configuración y Modelos de IA:**
    *   Configura el uso de la **GPU** (si está disponible) para acelerar drásticamente el procesamiento de grandes volúmenes de texto.
    *   Define dos modelos de Inteligencia Artificial clave:
        *   **`clapAI/roberta-large-multilingual-sentiment`**: Un modelo pre-entrenado (basado en la arquitectura RoBERTa) especializado en clasificar el sentimiento (positivo, negativo, neutro) de textos en múltiples idiomas, incluyendo español. Es "large" (grande), lo que implica que tiene muchos parámetros y es capaz de capturar matices complejos en el lenguaje.
        *   **`sentence-transformers/paraphrase-multilingual-mpnet-base-v2`**: Un modelo de "sentence-transformers" que genera **embeddings** (representaciones numéricas densas) de frases y párrafos. Es "multilingual" (multilingüe) y "paraphrase" (paráfrasis), lo que significa que puede entender la similitud semántica entre textos en diferentes idiomas o que expresan la misma idea de forma diferente. Es fundamental para el agrupamiento y la clasificación de temas por similitud de significado, no solo por palabras clave.

2.  **Diccionarios y Patrones Contextuales:**
    *   **`TOPIC_KEYWORDS`**: Un diccionario extenso con palabras clave primarias y secundarias asociadas a diferentes temas relevantes en el contexto colombiano (Política, Economía, Seguridad, etc.). Esto permite una identificación de temas más precisa y contextualizada.
    *   **`SENTIMENT_PATTERNS`**: Patrones de expresiones y palabras que indican fuertemente un sentimiento particular (positivo o negativo). Se usan para refinar el análisis de sentimiento, especialmente en textos cortos o con lenguaje muy directo.
    *   **`SENSITIVE_TOPICS`**: Define temas que, por su naturaleza (ej. violencia, derechos humanos), suelen tener un tono predominantemente negativo o neutro en el contexto de noticias. Se usa en el post-procesamiento para verificar la consistencia.

3.  **Utilidades de Limpieza y Normalización:**
    *   Funciones como `clean_text_enhanced` y `normalize_for_comparison` preparan los textos, eliminando ruido (URLs, caracteres especiales), manejando puntuación y acortando inteligentemente los textos muy largos sin perder el sentido principal. `get_text_signature` crea una "firma" corta para comparaciones rápidas.

4.  **Agrupamiento Inteligente Avanzado (`agrupar_textos_avanzado`):**
    *   Este es un proceso multi-etapa:
        *   **Nivel 1 (Firma Textual):** Agrupa inicialmente textos con inicios de frase muy similares para una pre-selección rápida.
        *   **Nivel 2 (Clustering Semántico):** Para los grupos de firmas, utiliza los embeddings del modelo `paraphrase-multilingual-mpnet-base-v2` y un algoritmo de clustering jerárquico (`AgglomerativeClustering` o `DBSCAN`) para agrupar textos basados en su similitud de significado, no solo de palabras. Identifica representantes para cada cluster.
        *   **Nivel 3 (Fusión de Micro-clusters):** Revisa los clusters resultantes y fusiona aquellos que son extremadamente similares semánticamente, reduciendo el número final de clusters y mejorando la organización.

5.  **Análisis de Sentimiento Avanzado (`inferir_sentimiento_avanzado`):**
    *   Utiliza el modelo `roberta-large-multilingual-sentiment` para obtener un sentimiento base para los textos representantes de cada cluster.
    *   Implementa un **sistema de votación** para los clusters con múltiples textos. Analiza los patrones de sentimiento en varias muestras dentro del cluster y combina estos resultados con el sentimiento base del representante para determinar el tono final del cluster. Esto mejora la robustez del análisis para grupos de textos.

6.  **Clasificación de Temas Avanzada (`inferir_temas_avanzado`):**
    *   Combina el análisis de **keywords** definidas en `TOPIC_KEYWORDS` con la **similitud semántica** calculada con los embeddings.
    *   Prioriza los matches fuertes de keywords, pero utiliza la similitud con "centroides" temáticos (frases que representan cada tema) para textos donde las keywords no son tan claras o para refinar la asignación.

7.  **Post-procesamiento de Consistencia (`post_procesar_consistencia`):**
    *   Ajusta los resultados de tono y tema basándose en las reglas definidas en `SENSITIVE_TOPICS` y otras inconsistencias detectadas. Por ejemplo, si un texto sobre un tema sensible (como una masacre) fue clasificado erróneamente como "Positivo", este paso puede corregirlo a "Neutro" o "Negativo" si hay suficiente evidencia contextual.

8.  **Análisis de Calidad (`analizar_calidad_resultados`):**
    *   Proporciona métricas sobre la calidad de los resultados, como la consistencia del tono y tema dentro de los clusters, la distribución de tamaños de los clusters y el balance general de tonos y temas identificados.

9.  **Exportación Avanzada (`guardar_resultados_avanzado`):**
    *   Genera un archivo Excel detallado con los resultados para cada texto original (incluyendo su tono, tema, ID de cluster y tamaño del cluster).
    *   Incluye hojas adicionales con estadísticas generales, distribuciones de tono y tema, un cruce entre tema y tono, un análisis de los clusters y ejemplos de textos por tema. Esto facilita la interpretación y el uso posterior de los resultados.

10. **Flujo Principal (`main`):**
    *   Coordina todo el proceso. Solicita al usuario que suba un archivo Excel con una columna llamada "resumen", ejecuta las fases de agrupamiento, análisis de sentimiento, clasificación de temas y post-procesamiento, y finalmente guarda y descarga el archivo de resultados.

**Ventajas de esta Implementación:**

*   **Precisión Contextual:** El uso de diccionarios y reglas específicas para el contexto colombiano mejora la relevancia y precisión de los temas y tonos identificados.
*   **Robustez con Agrupamiento:** Al analizar clusters de textos similares en lugar de cada texto individualmente para el sentimiento, se reduce el ruido y se obtiene un tono más representativo para eventos o ideas recurrentes.
*   **Análisis Multi-método:** La combinación de análisis basado en keywords y análisis semántico con embeddings proporciona una clasificación de temas más confiable.
*   **Post-procesamiento Inteligente:** Los ajustes basados en temas sensibles y patrones contextuales corrigen posibles errores de los modelos de IA en casos específicos.
*   **Resultados Detallados y Organizados:** El archivo de salida incluye no solo el resultado por texto, sino también estadísticas y análisis a nivel de cluster y general, lo que facilita la exploración de los datos.
*   **Optimización de Recursos:** El uso de procesamiento por lotes (batch processing) y la gestión de memoria (limpieza de caché de GPU) mejoran la eficiencia, especialmente con grandes conjuntos de datos.

En resumen, esta versión del analizador va más allá de una simple clasificación individual de textos, aplicando inteligencia para agrupar información similar y refinar los resultados basándose en el contexto y patrones específicos.